# RunnableLambda

RunnableLambda 将一个普通函数时转换为 Runnable 对象。

## 使用invoke和batch

In [85]:
from langchain_core.runnables import RunnableLambda
import time

# 定义你的函数
def hello(name):
    return f"hi {name} ! Welcome To Langchain!"

# 创建一个MyRunnableLambda实例
runnable = RunnableLambda(hello)

# 使用invoke获得一次性输出
print(runnable.invoke("Alice"))  # 输出 "hi Alice !"

# 注意：因为hello函数没有流式输出，因此stream也是没有流式输出效果的
for output in runnable.stream("Alice"):
    time.sleep(0.3)
    print(output)  # 输出 "hi Alice !"

hi Alice ! Welcome To Langchain!
hi Alice ! Welcome To Langchain!


## 使用batch获得批量结果

In [89]:
# batch中会自动调用invoke
runnable.batch(["Alice", "Bob"])

['hi Alice ! Welcome To Langchain!', 'hi Bob ! Welcome To Langchain!']

## 获得stream的流式输出效果

In [90]:
from langchain_core.runnables import RunnableLambda
import time

# 定义你的函数
def hello(name):
    yield "hi "
    yield name
    yield "! "
    yield "Welcome"
    yield " to"
    yield " Langchain!"

# 创建一个MyRunnableLambda实例
runnable = RunnableLambda(hello)

# invoke会等到所有结果获得后合并到一起输出
print(runnable.invoke("Alice"))  # 输出 "hi Alice !"

# 因为hello函数实现了yield，stream就会有流式输出效果
for output in runnable.stream("Alice"):
    time.sleep(0.3)
    print(output, end = "")  # 输出 "hi Alice !"

hi Alice! Welcome to Langchain!
hi Alice! Welcome to Langchain!

## 使用 @chain 达到同样的效果

In [91]:
from langchain_core.runnables import chain
import time

# 定义你的函数
@chain
def hello(name):
    yield "hi "
    yield name
    yield "! "
    yield "Welcome"
    yield " to"
    yield " Langchain!"

print(hello.invoke("Alice"))  # 输出 "hi Alice !"

for output in hello.stream("Alice"):
    time.sleep(0.3)
    print(output, end = "")  # 输出 "hi Alice !"

hi Alice! Welcome to Langchain!
hi Alice! Welcome to Langchain!

# RunnableGenerator

RunnableGenerator是专门用于生成器优化的实现，在将链作为输入时无需特别指定即可使用。

In [149]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableGenerator
from langchain_openai import ChatOpenAI
from langchain.schema import StrOutputParser
from typing import Iterator

model = ChatOpenAI()
chant = (
    ChatPromptTemplate.from_template("Give me a 3 word chant about {topic}")
    | model
    | StrOutputParser()
)

def character_generator(input: Iterator[str]) -> Iterator[str]:
    for token in input:
        if "," in token or "." in token:
            yield "👏" + token
        else:
            yield token

runnable = chant | character_generator
assert type(runnable.last) is RunnableGenerator
"".join(runnable.stream({"topic": "waste"})) # Reduce👏, Reuse👏, Recycle👏.

'Reduce👏, reuse👏, recycle!'